In [ ]:
#!pip install -U tensorflow-addons
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import os
import gc
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import os
import sys
import random
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import time
#import cv2 as cv
from tensorflow.keras.callbacks import *
from tensorflow.python.saved_model import loader_impl # lib for saving models
import plotly.graph_objects as go
#import seaborn as sns
#import pandas as pd
import random; random.seed(2)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self) -> None:
        super().__init__()

    #ef on_train_batch_begin(self, batch, logs = None):
    #    #lr = k.get_values(self.model.optimizer.lr)
    #    lr = tf.keras.backend.get_value(
    #        self.model.optimizer.lr(self.model.optimizer.iterations))
    #    #print(self.model.optimizer._decayed_lr('float32').numpy())
    #    print('\nLearning rate:',lr)
    #    #tf.summary.scalar('learning rate', data=lr, step=epoch)

class printlearningrate(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = 10#float(K.get_value(optimizer.lr))
        Epoch_count = epoch + 1

        #print('\n Learning Rate:',lr)
        #print('\n', "Epoch:", Epoch_count, ', LR: {:.2f}'.format(lr))

<h1><center> UNET Base </center></h1>

In [ ]:
class UNET_BASE(CustomCallback, printlearningrate):
    def __init__(self, optimizer, activation_function, initializer, inputs, initial_filter =16):
        self.optimizer           = optimizer
        self.activation_function = activation_function
        self.initializer         = initializer
        self.initial_filter      = initial_filter
        self.inputs              = inputs

    def Unet(self):
        inputs = self.inputs
        initializer    = self.initializer
        initial_filter = self.initial_filter
        activation_func = self.activation_function
        s  = tf.keras.layers.Lambda(lambda x: x)(inputs) # El maximo valor es 505
        c1 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c1_1' )(s)
        c1 = tf.keras.layers.Dropout(0.1)(c1) #Original 0.1
        c1 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c1_2' )(c1)
        p1 = tf.keras.layers.MaxPool2D((2,2))(c1)

        c2 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c2_1')(p1)
        c2 = tf.keras.layers.Dropout(0.1)(c2) #original 0.1
        ##c2 = tf.keras.layers.Dropout(general_dropOut)(c2) #Orignal 0.1
        c2 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c2_2')(c2)
        p2 = tf.keras.layers.MaxPool2D((2,2))(c2)

        c3 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c3_1')(p2)
        c3 = tf.keras.layers.Dropout(0.2)(c3)#original 0.2
        c3 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c3_2')(c3)
        p3 = tf.keras.layers.MaxPool2D((2,2))(c3)

        c4 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c4_1')(p3)
        c4 = tf.keras.layers.Dropout(0.2)(c4)#original 0.2
        c4 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c4_2')(c4)
        p4 = tf.keras.layers.MaxPool2D((2,2))(c4)

        c5 = tf.keras.layers.Conv2D(initial_filter*16, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c5_1')(p4)
        c5 = tf.keras.layers.Dropout(0.3)(c5)#original 0.3
        c5 = tf.keras.layers.Conv2D(initial_filter*16, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c5_2')(c5)

        #space latent
        u6 = tf.keras.layers.Conv2DTranspose(initial_filter*8, (2,2), strides=(2,2), padding='same', name='u6')(c5)
        u6 = tf.keras.layers.concatenate([u6, c4])
        c6 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c6_1')(u6)
        c6 = tf.keras.layers.Dropout(0.2)(c6) #original 0.2
        c6 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c6_2')(c6)

        u7 = tf.keras.layers.Conv2DTranspose(initial_filter*4, (2,2), strides=(2,2), padding='same')(c6)
        u7 = tf.keras.layers.concatenate([u7, c3])
        c7 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u7)
        c7 = tf.keras.layers.Dropout(0.2)(c7)# original 0.2
        c7 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c7)

        u8 = tf.keras.layers.Conv2DTranspose(initial_filter*2, (2,2), strides=(2,2), padding='same')(c7)
        u8 = tf.keras.layers.concatenate([u8, c2])
        c8 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u8)
        c8 = tf.keras.layers.Dropout(0.1)(c8) #original 0.1
        c8 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c8)

        u9 = tf.keras.layers.Conv2DTranspose(initial_filter, (2,2), strides=(2,2), padding='same')(c8)
        u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
        c9 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u9)
        c9 = tf.keras.layers.Dropout(0.1)(c9)#original 0.1
        c9 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c9)

        outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)
        Model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
        return Model



    def train(self, X_train_, Y_train_, btz, epochs, min_lr = 0.001, max_lr = 0.8):

        INIT_LR    = min_lr #1e-6  0.001
        MAX_LR     = max_lr   #1e-4
        steps_per_epoch = len(X_train_) // btz
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                    filepath          = 'test',
                                    verbose           = True,
                                    save_weights_only = True,
                                    monitor           = 'val_loss',
                                    mode              = 'max',
                                    save_best_only=True)
        self.Model = self.Unet()
        self.Model.compile(optimizer=self.optimizer, loss='binary_crossentropy',
                           metrics=['accuracy'])
        results = self.Model.fit(X_train_,Y_train_, validation_split=0.20, batch_size=btz,
                                 epochs = epochs,
                                 callbacks=[model_checkpoint_callback,CustomCallback(), printlearningrate()])

    def testing(self, X_test_, Y_test_, th = 0.5):
        metrics = []
        self.iou_score   = 0
        self.dc          = 0.0
        self.sensitivity = 0
        self.specificity = 0
        SM          = Semantic_loss_functions()
        #print(np.shape(Y_test_))
        for i in np.arange(len(X_test_)):
            img_  = resize(X_test_[i], (240, 240), mode='constant', preserve_range=True)
            if np.min(X_test_[i]) == np.max(X_test_[i]):
                continue
            img_ = np.expand_dims(img_, axis=0)
            y_p  = self.Model.predict(img_)
            im_p = (y_p>= th).astype(np.uint8)
            im_p = np.squeeze(im_p)
            y_test = np.uint8(Y_test_[i])
            intersection = np.logical_and( y_test, im_p)
            union = np.logical_or(y_test, im_p)
            #print('Interseccion',np.sum(intersection))
            if np.sum(union) > 0:
                self.iou_score += np.sum(intersection) / ( np.sum(union))
            else:
                self.iou_score += 0
            #print('np.sum(union)',np.sum(union))

            im_p         = im_p.astype(np.float32)
            y_test       = y_test.astype(np.float32)
            self.dc          += tf.get_static_value(SM.dice_coef(y_test, im_p) )
            self.sensitivity += tf.get_static_value(SM.sensitivity(y_test, im_p ))
            self.specificity += tf.get_static_value(SM.specificity(y_test, im_p ))

            #iou_score= iou_score/len(X_test_)
        self.IOU         = np.float32(self.iou_score/(len(X_test_)))
        self.dc          = np.float32( self.dc /(len(X_test_)))
        self.sensitivity = self.sensitivity / (len(X_test_))
        self.specificity =  self.specificity / (len(X_test_))
        #print('IOU score:', self.iou_score/(len(X_test_)))
        metrics.append(self.IOU)
        metrics.append(self.dc)
        metrics.append(self.sensitivity)
        metrics.append(self.specificity)
        return metrics

    def get_model(self):
        return self.Model

In [ ]:
class UNET(CustomCallback, printlearningrate):
    def __init__(self, optimizer, activation_function, initializer, inputs, initial_filter =16):
        self.optimizer           = optimizer
        self.activation_function = activation_function
        self.initializer         = initializer
        self.initial_filter      = initial_filter
        self.inputs              = inputs
        self.history = 0

    def Unet(self):
        inputs = self.inputs
        initializer    = self.initializer
        initial_filter = self.initial_filter
        activation_func = self.activation_function
        s  = tf.keras.layers.Lambda(lambda x: x)(inputs) # El maximo valor es 505
        c1 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c1_1' )(s)
        c1 = tf.keras.layers.BatchNormalization()(c1)
        c1 = tf.keras.layers.Dropout(0.1)(c1) #Original 0.1
        c1 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c1_2' )(c1)
        c1 = tf.keras.layers.BatchNormalization()(c1)
        p1 = tf.keras.layers.MaxPool2D((2,2))(c1)

        c2 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c2_1')(p1)
        c2 = tf.keras.layers.BatchNormalization()(c2)
        c2 = tf.keras.layers.Dropout(0.1)(c2) #original 0.1
        c2 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c2_2')(c2)
        c2 = tf.keras.layers.BatchNormalization()(c2)
        p2 = tf.keras.layers.MaxPool2D((2,2))(c2)

        c3 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c3_1')(p2)
        c3 = tf.keras.layers.BatchNormalization()(c3)
        c3 = tf.keras.layers.Dropout(0.2)(c3)#original 0.2
        c3 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c3_2')(c3)
        c3 = tf.keras.layers.BatchNormalization()(c3)
        p3 = tf.keras.layers.MaxPool2D((2,2))(c3)

        c4 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c4_1')(p3)
        c4 = tf.keras.layers.BatchNormalization()(c4)
        c4 = tf.keras.layers.Dropout(0.2)(c4)#original 0.2
        c4 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c4_2')(c4)
        c4 = tf.keras.layers.BatchNormalization()(c4)
        p4 = tf.keras.layers.MaxPool2D((2,2))(c4)

        c5 = tf.keras.layers.Conv2D(initial_filter*16, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c5_1')(p4)
        c5 = tf.keras.layers.BatchNormalization()(c5)
        c5 = tf.keras.layers.Dropout(0.3)(c5)#original 0.3
        c5 = tf.keras.layers.Conv2D(initial_filter*16, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c5_2')(c5)
        c5 = tf.keras.layers.BatchNormalization()(c5)

        #space latent
        u6 = tf.keras.layers.Conv2DTranspose(initial_filter*8, (2,2), strides=(2,2), padding='same', name='u6')(c5)
        u6 = tf.keras.layers.concatenate([u6, c4])
        c6 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c6_1')(u6)
        c6 = tf.keras.layers.BatchNormalization()(c6)
        c6 = tf.keras.layers.Dropout(0.2)(c6) #original 0.2
        c6 = tf.keras.layers.Conv2D(initial_filter*8, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same', name='c6_2')(c6)
        c6 = tf.keras.layers.BatchNormalization()(c6)

        u7 = tf.keras.layers.Conv2DTranspose(initial_filter*4, (2,2), strides=(2,2), padding='same')(c6)
        u7 = tf.keras.layers.concatenate([u7, c3])
        c7 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u7)
        c7 = tf.keras.layers.BatchNormalization()(c7)
        c7 = tf.keras.layers.Dropout(0.2)(c7)# original 0.2
        c7 = tf.keras.layers.Conv2D(initial_filter*4, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c7)
        c7 = tf.keras.layers.BatchNormalization()(c7)

        u8 = tf.keras.layers.Conv2DTranspose(initial_filter*2, (2,2), strides=(2,2), padding='same')(c7)
        u8 = tf.keras.layers.BatchNormalization()(u8)
        u8 = tf.keras.layers.concatenate([u8, c2])
        c8 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u8)
        c8 = tf.keras.layers.BatchNormalization()(c8)
        c8 = tf.keras.layers.Dropout(0.1)(c8) #original 0.1
        c8 = tf.keras.layers.Conv2D(initial_filter*2, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c8)
        c8 = tf.keras.layers.BatchNormalization()(c8)

        u9 = tf.keras.layers.Conv2DTranspose(initial_filter, (2,2), strides=(2,2), padding='same')(c8)
        u9 = tf.keras.layers.BatchNormalization()(u9)
        u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
        c9 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(u9)
        c9 = tf.keras.layers.BatchNormalization()(c9)
        c9 = tf.keras.layers.Dropout(0.1)(c9)#original 0.1
        c9 = tf.keras.layers.Conv2D(initial_filter, (3,3), activation=activation_func, kernel_initializer=initializer, padding='same')(c9)
        c9 = tf.keras.layers.BatchNormalization()(c9)

        outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)
        Model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
        return Model

    def train(self, X_train_, Y_train_, epochs, btz, X_test_ = None, Y_test_= None, min_lr = 0.001, max_lr = 0.8, show_history=True):

        INIT_LR    = min_lr #1e-6  0.001
        MAX_LR     = max_lr   #1e-4
        steps_per_epoch = len(X_train_) // btz
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                    filepath          = 'test',
                                    verbose           = True,
                                    save_weights_only = True,
                                    monitor           = 'val_loss',
                                    mode              = 'min',
                                    save_best_only=True)
        self.Model = self.Unet()
        self.Model.compile(optimizer=self.optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        if X_test_ != None and Y_test_ != None:
          self.history = self.Model.fit(X_train_,Y_train_, validation_data=(X_test_, Y_test_), batch_size=btz,
                                 epochs = epochs, callbacks=[model_checkpoint_callback,CustomCallback(), printlearningrate()])
        else:
          self.history = self.Model.fit(X_train_,Y_train_, validation_split=0.20, batch_size=btz,
                                 epochs = epochs, callbacks=[model_checkpoint_callback,CustomCallback(), printlearningrate()])
        if show_history == True:
            print(self.history.history.keys())  # Ver las métricas disponibles en el historial
            # Graficar la pérdida del entrenamiento y la validación
            plt.plot(self.history.history['loss'], label='Pérdida de entrenamiento')
            plt.plot(self.history.history['val_loss'], label='Pérdida de validación')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.legend()
            plt.show()

            # Graficar la precisión del entrenamiento y la validación
            plt.plot(self.history.history['accuracy'], label='accuracy training')
            plt.plot(self.history.history['val_accuracy'], label='accuracy validation')
            plt.xlabel('Epochs')
            plt.ylabel('Precision')
            plt.legend()
            plt.show()

    def testing(self, X_test_, Y_test_, th = 0.5):
        metrics = []
        self.iou_score   = 0
        self.dc          = 0.0
        self.sensitivity = 0
        self.specificity = 0
        SM          = Semantic_loss_functions()
        #print(np.shape(Y_test_))
        for i in np.arange(len(X_test_)):
            img_  = resize(X_test_[i], (240, 240), mode='constant', preserve_range=True)
            if np.min(X_test_[i]) == np.max(X_test_[i]):
                continue
            img_ = np.expand_dims(img_, axis=0)
            y_p  = self.Model.predict(img_)
            im_p = (y_p>= th).astype(np.uint8)
            im_p = np.squeeze(im_p)
            y_test = np.uint8(Y_test_[i])
            intersection = np.logical_and( y_test, im_p)
            union = np.logical_or(y_test, im_p)
            #print('Interseccion',np.sum(intersection))
            if np.sum(union) > 0:
                self.iou_score += np.sum(intersection) / ( np.sum(union))
            else:
                self.iou_score += 0
            #print('np.sum(union)',np.sum(union))

            im_p         = im_p.astype(np.float32)
            y_test       = y_test.astype(np.float32)
            self.dc          += tf.get_static_value(SM.dice_coef(y_test, im_p) )
            self.sensitivity += tf.get_static_value(SM.sensitivity(y_test, im_p ))
            self.specificity += tf.get_static_value(SM.specificity(y_test, im_p ))

            #iou_score= iou_score/len(X_test_)
        self.IOU         = np.float32(self.iou_score/(len(X_test_)))
        self.dc          = np.float32( self.dc /(len(X_test_)))
        self.sensitivity = self.sensitivity / (len(X_test_))
        self.specificity =  self.specificity / (len(X_test_))
        #print('IOU score:', self.iou_score/(len(X_test_)))
        metrics.append(self.IOU)
        metrics.append(self.dc)
        metrics.append(self.sensitivity)
        metrics.append(self.specificity)
        return metrics

    def get_model(self):
        return self.Model

    def get_history(self):
        return self.history

    def dice_coefficient(self, y_true, y_pred, smooth=1e-6):
        y_true_f = tf.keras.backend.flatten(y_true)
        y_pred_f = tf.keras.backend.flatten(y_pred)
        intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

    def dice_loss(self, y_true, y_pred, smooth=1e-6):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Asegúrate de que y_true y y_pred tienen la misma forma
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
        union = tf.reduce_sum(y_true, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2])
        dice = (2. * intersection + smooth) / (union + smooth)

        # Devolvemos 1 menos la media del coeficiente Dice
        return 1 - tf.reduce_mean(dice)

    def cross_entropy_loss(self, y_true, y_pred):
        return tf.keras.losses.binary_crossentropy(y_true, y_pred)

    def combined_loss(self, y_true, y_pred):
        # Pérdida Dice
        dice_loss_value = self.dice_loss(y_true, y_pred)

        # Pérdida de entropía cruzada binaria
        bce_loss_value = self.cross_entropy_loss(y_true, y_pred)

        # Combinación ponderada de las pérdidas
        total_loss = 0.5 * dice_loss_value + 0.5 * bce_loss_value
        return total_loss

<h1><center> UNET ++ </center></h1>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt

class UNETPP:
    def __init__(self, input_shape=(240, 240, 1), num_filters=64, num_classes=1):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.num_classes = num_classes

    def get_model(self):
        return self.model

    def conv_block(self, input_tensor, num_filters):
        x = layers.Conv2D(num_filters, 3, padding='same')(input_tensor)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(num_filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    def dense_concat(self, blocks):
        return layers.Concatenate()(blocks)

    def build(self):
        inputs = Input(shape=self.input_shape)

        # Encoder
        c1 = self.conv_block(inputs, self.num_filters)
        p1 = layers.MaxPooling2D((2, 2))(c1)

        c2 = self.conv_block(p1, self.num_filters * 2)
        p2 = layers.MaxPooling2D((2, 2))(c2)

        c3 = self.conv_block(p2, self.num_filters * 4)
        p3 = layers.MaxPooling2D((2, 2))(c3)

        c4 = self.conv_block(p3, self.num_filters * 8)
        p4 = layers.MaxPooling2D((2, 2))(c4)

        # Bridge
        bridge = self.conv_block(p4, self.num_filters * 16)

        # Decoder with dense connections
        u4 = layers.UpSampling2D((2, 2))(bridge)
        u4 = self.dense_concat([u4, c4])
        c4d = self.conv_block(u4, self.num_filters * 8)

        u3 = layers.UpSampling2D((2, 2))(c4d)
        u3 = self.dense_concat([u3, c3])
        c3d = self.conv_block(u3, self.num_filters * 4)

        u2 = layers.UpSampling2D((2, 2))(c3d)
        u2 = self.dense_concat([u2, c2])
        c2d = self.conv_block(u2, self.num_filters * 2)

        u1 = layers.UpSampling2D((2, 2))(c2d)
        u1 = self.dense_concat([u1, c1])
        c1d = self.conv_block(u1, self.num_filters)

        outputs = layers.Conv2D(self.num_classes, 1, activation='sigmoid')(c1d)

        model = models.Model(inputs=[inputs], outputs=[outputs])
        return model

    def train(self, X_train, Y_train, batch_size, epochs, optimizer='adam', X_test=None, Y_test=None):
        self.model = self.build()
        self.model.summary()
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        if X_test is not None and Y_test is not None:
            self.history = self.model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
                                    batch_size=batch_size, epochs=epochs)
        else:
            self.history = self.model.fit(X_train, Y_train, validation_split=0.2, batch_size=batch_size, epochs=epochs)
        return self.model

    def testing(self, X_test, Y_test, th=0.5, show_images=False):
        metrics = []
        iou_score = 0
        dc = 0.0
        sensitivity = 0
        specificity = 0
        SM = Semantic_loss_functions()

        for i in np.arange(len(X_test)):
            img_ = resize(X_test[i], (240, 240), mode='constant', preserve_range=True)
            if np.min(X_test[i]) == np.max(X_test[i]):
                continue
            img_ = np.expand_dims(img_, axis=0)
            y_p = self.model.predict(img_)
            im_p = (y_p >= th).astype(np.uint8)
            im_p = np.squeeze(im_p)
            y_test = np.uint8(Y_test[i])
            if show_images:
                plt.subplot(1, 2, 1), plt.imshow(im_p, cmap='gray')
                plt.subplot(1, 2, 2), plt.imshow(y_test, cmap='gray')
                plt.show()
            intersection = np.logical_and(y_test, im_p)
            union = np.logical_or(y_test, im_p)
            if np.sum(union) > 0:
                iou_score += np.sum(intersection) / np.sum(union)
            else:
                iou_score += 0
            im_p = im_p.astype(np.float32)
            y_test = y_test.astype(np.float32)
            dc += tf.get_static_value(SM.dice_coef(y_test, im_p))
            sensitivity += tf.get_static_value(SM.sensitivity(y_test, im_p))
            specificity += tf.get_static_value(SM.specificity(y_test, im_p))

        IOU = np.float32(iou_score / len(X_test))
        dc = np.float32(dc / len(X_test))
        sensitivity = sensitivity / len(X_test)
        specificity = specificity / len(X_test)
        metrics.append(IOU)
        metrics.append(dc)
        metrics.append(sensitivity)
        metrics.append(specificity)
        return metrics
    def get_model(self):
        return self.model
    def get_history(self):
        return self.history

In [ ]:
#https://github.com/shruti-jadon/Semantic-Segmentation-Loss-Functions
import tensorflow as tf
import tensorflow.keras.backend as K
#from keras.losses import binary_crossentropy

beta = 0.25
alpha = 0.25
gamma = 2
epsilon = 1e-5
smooth = 1


class Semantic_loss_functions(object):
    def __init__(self):
        print ("semantic loss functions initialized")

    def dice_coef(self, y_true, y_pred):
        #intersection = np.sum(y_true * y_pred)
        #return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        #print('K epsilon',K.epsilon())
        return (2. * intersection + 1) / (
                    K.sum(y_true_f) + K.sum(y_pred_f) + epsilon)# 1)

    def sensitivity(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        return true_positives / (possible_positives + K.epsilon())

    def specificity(self, y_true, y_pred):
        true_negatives = K.sum(
            K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
        possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
        return true_negatives / (possible_negatives + K.epsilon())